<a href="https://colab.research.google.com/github/sYanXO/NSE-meanReversion-strategy/blob/main/Mean_Reversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- INSTALL AND IMPORT LIBRARIES ---
!pip install ta
!pip install yfinance

import yfinance as yf
import pandas as pd
import ta
import matplotlib.pyplot as plt

# --- STEP 1: DEFINE PORTFOLIO AND FIXED PARAMETERS ---

# A large sample of tickers from major NSE indices
tickers = [
    "RELIANCE.NS", "ADANIENT.NS", "HDFCBANK.NS", "ICICIBANK.NS", "INFY.NS", "TCS.NS",
    "LT.NS", "AXISBANK.NS", "BHARTIARTL.NS", "HINDUNILVR.NS", "KOTAKBANK.NS",
    "ITC.NS", "SBIN.NS", "MARUTI.NS", "BAJFINANCE.NS", "ASIANPAINT.NS",
    "TITAN.NS", "WIPRO.NS", "NESTLEIND.NS", "ULTRACEMCO.NS", "NTPC.NS",
    "POWERGRID.NS", "TATAMOTORS.NS", "TECHM.NS", "BRITANNIA.NS", "ONGC.NS",
    "JSWSTEEL.NS", "HCLTECH.NS", "GRASIM.NS", "EICHERMOT.NS", "HEROMOTOCO.NS",
    "ADANIPORTS.NS", "COALINDIA.NS", "APOLLOHOSP.NS", "CIPLA.NS", "DRREDDY.NS",
    "INDUSINDBK.NS", "M&M.NS", "SBILIFE.NS", "TATASTEEL.NS", "TATACONSUM.NS",
    "HDFCBANK.NS", "ADANIENSOL.NS", "ADANIGREEN.NS", "ASIANPAINT.NS",
    "COFORGE.NS", "GODREJCP.NS", "HINDALCO.NS", "INFY.NS", "LTIM.NS",
    "PIDILITIND.NS", "PIIND.NS", "RECLTD.NS", "SBICARD.NS", "SIEMENS.NS",
    "SRF.NS", "SYNGENE.NS", "TATACOMM.NS", "TVSMOTOR.NS", "UNIONBANK.NS",
    "VOLTAS.NS"
]

start_date = "2020-01-01"
end_date = "2024-07-31"

# The optimal parameters we found through our iterative process
window = 20
long_term_window = 100
atr_multiple = 3.0
nifty_ma_window = 200
transaction_cost_percent = 0.001
slippage_percent = 0.001

# --- STEP 2: OPTIMIZATION LOOP FOR TREND_FILTER_THRESHOLD ---

# Create a DataFrame to store our results
optimization_results = pd.DataFrame(columns=['Trend_Filter', 'Cumulative_Return', 'Total_Trades', 'Win_Rate'])

# Define a range of values to test for the trend_filter_threshold
test_thresholds = [0.01, 0.02, 0.03, 0.04, 0.05]

for trend_filter_threshold in test_thresholds:
    print(f"\n--- Testing with Trend Filter Threshold: {trend_filter_threshold} ---")

    # Re-initialize the portfolio returns list for each run
    all_portfolio_returns = []

    # --- NIFTY 50 DATA DOWNLOAD ---
    try:
        nifty_df = yf.download('^NSEI', start=start_date, end=end_date, progress=False)
        nifty_df.columns = nifty_df.columns.droplevel(1)
        nifty_df['Nifty_MA'] = nifty_df['Close'].rolling(window=nifty_ma_window).mean()
        nifty_df.dropna(inplace=True)
    except Exception as e:
        print(f"Error downloading Nifty 50 data: {e}. Skipping threshold {trend_filter_threshold}.")
        continue

    # --- MAIN BACKTESTING LOOP ---
    for i, ticker in enumerate(tickers):
        try:
            df = yf.download(ticker, start=start_date, end=end_date, progress=False)
            if df.empty:
                continue
            df.columns = df.columns.droplevel(1)
        except Exception as e:
            continue

        df['SMA'] = df['Close'].rolling(window=window).mean()
        df['StdDev'] = df['Close'].rolling(window=window).std()
        df['Upper_Band'] = df['SMA'] + (df['StdDev'] * 2)
        df['Lower_Band'] = df['SMA'] - (df['StdDev'] * 2)
        df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()
        df['Long_SMA'] = df['Close'].rolling(window=long_term_window).mean()
        df['ATR'] = ta.volatility.AverageTrueRange(high=df['High'], low=df['Low'], close=df['Close'], window=14).average_true_range()

        df = df.join(nifty_df[['Nifty_MA', 'Close']], how='inner', lsuffix='_stock', rsuffix='_nifty')
        df.rename(columns={'Close_nifty': 'Nifty_Close', 'Close_stock': 'Close'}, inplace=True)
        df.dropna(inplace=True)

        in_long_position = False
        in_short_position = False
        long_buy_price = 0
        short_sell_price = 0

        for j in range(len(df)):
            close_price = df['Close'][j]
            upper_band = df['Upper_Band'][j]
            lower_band = df['Lower_Band'][j]
            sma = df['SMA'][j]
            long_sma = df['Long_SMA'][j]
            rsi = df['RSI'][j]
            atr = df['ATR'][j]
            nifty_ma = df['Nifty_MA'][j]
            nifty_close = df['Nifty_Close'][j]

            is_trending_up = close_price > long_sma * (1 + trend_filter_threshold)
            is_trending_down = close_price < long_sma * (1 - trend_filter_threshold)

            nifty_is_bullish = nifty_close > nifty_ma
            nifty_is_bearish = nifty_close < nifty_ma

            # --- Long Position Logic ---
            if in_long_position and (close_price <= long_buy_price - (atr * atr_multiple)):
                profit = ((close_price - long_buy_price) / long_buy_price) - (2 * transaction_cost_percent) - (2 * slippage_percent)
                all_portfolio_returns.append(profit)
                in_long_position = False
            elif not in_long_position and not in_short_position and not is_trending_down and nifty_is_bullish and close_price < lower_band and rsi < 30:
                in_long_position = True
                long_buy_price = close_price
            elif in_long_position and close_price >= sma:
                profit = ((close_price - long_buy_price) / long_buy_price) - (2 * transaction_cost_percent) - (2 * slippage_percent)
                all_portfolio_returns.append(profit)
                in_long_position = False

            # --- Short Position Logic ---
            if in_short_position and (close_price >= short_sell_price + (atr * atr_multiple)):
                profit = ((short_sell_price - close_price) / short_sell_price) - (2 * transaction_cost_percent) - (2 * slippage_percent)
                all_portfolio_returns.append(profit)
                in_short_position = False
            elif not in_long_position and not in_short_position and not is_trending_up and nifty_is_bearish and close_price > upper_band and rsi > 70:
                in_short_position = True
                short_sell_price = close_price
            elif in_short_position and close_price <= sma:
                profit = ((short_sell_price - close_price) / short_sell_price) - (2 * transaction_cost_percent) - (2 * slippage_percent)
                all_portfolio_returns.append(profit)
                in_short_position = False

        if in_long_position:
            profit = ((df['Close'].iloc[-1] - long_buy_price) / long_buy_price) - transaction_cost_percent - slippage_percent
            all_portfolio_returns.append(profit)
        if in_short_position:
            profit = ((short_sell_price - df['Close'].iloc[-1]) / short_sell_price) - transaction_cost_percent - slippage_percent
            all_portfolio_returns.append(profit)

    # After the backtest loop finishes for this threshold:
    if all_portfolio_returns:
        total_trades = len(all_portfolio_returns)
        cumulative_return = (pd.Series(all_portfolio_returns) + 1).prod() - 1
        win_rate = sum(r > 0 for r in all_portfolio_returns) / total_trades

        optimization_results = pd.concat([optimization_results, pd.DataFrame([{'Trend_Filter': trend_filter_threshold, 'Cumulative_Return': cumulative_return, 'Total_Trades': total_trades, 'Win_Rate': win_rate}])], ignore_index=True)
    else:
        optimization_results = pd.concat([optimization_results, pd.DataFrame([{'Trend_Filter': trend_filter_threshold, 'Cumulative_Return': 0.0, 'Total_Trades': 0, 'Win_Rate': 0.0}])], ignore_index=True)

# --- STEP 3: DISPLAY FINAL OPTIMIZATION RESULTS ---
print("\n\n--- Optimization Results ---")
print(optimization_results.to_string())

best_result = optimization_results.loc[optimization_results['Cumulative_Return'].idxmax()]
print("\nBest Performing Parameter:")
print(best_result.to_string())

In [ ]:
# --- INSTALL AND IMPORT LIBRARIES ---
!pip install ta
!pip install yfinance

import yfinance as yf
import pandas as pd
import ta
import matplotlib.pyplot as plt

# --- STEP 1: DEFINE PORTFOLIO AND FIXED PARAMETERS ---

# A large sample of tickers from major NSE indices
tickers = [
    "RELIANCE.NS", "ADANIENT.NS", "HDFCBANK.NS", "ICICIBANK.NS", "INFY.NS", "TCS.NS",
    "LT.NS", "AXISBANK.NS", "BHARTIARTL.NS", "HINDUNILVR.NS", "KOTAKBANK.NS",
    "ITC.NS", "SBIN.NS", "MARUTI.NS", "BAJFINANCE.NS", "ASIANPAINT.NS",
    "TITAN.NS", "WIPRO.NS", "NESTLEIND.NS", "ULTRACEMCO.NS", "NTPC.NS",
    "POWERGRID.NS", "TATAMOTORS.NS", "TECHM.NS", "BRITANNIA.NS", "ONGC.NS",
    "JSWSTEEL.NS", "HCLTECH.NS", "GRASIM.NS", "EICHERMOT.NS", "HEROMOTOCO.NS",
    "ADANIPORTS.NS", "COALINDIA.NS", "APOLLOHOSP.NS", "CIPLA.NS", "DRREDDY.NS",
    "INDUSINDBK.NS", "M&M.NS", "SBILIFE.NS", "TATASTEEL.NS", "TATACONSUM.NS",
    "HDFCBANK.NS", "ADANIENSOL.NS", "ADANIGREEN.NS", "ASIANPAINT.NS",
    "COFORGE.NS", "GODREJCP.NS", "HINDALCO.NS", "INFY.NS", "LTIM.NS",
    "PIDILITIND.NS", "PIIND.NS", "RECLTD.NS", "SBICARD.NS", "SIEMENS.NS",
    "SRF.NS", "SYNGENE.NS", "TATACOMM.NS", "TVSMOTOR.NS", "UNIONBANK.NS",
    "VOLTAS.NS"
]

start_date = "2020-01-01"
end_date = "2024-07-31"

# The optimal parameters we found through our iterative process
window = 20
long_term_window = 100
atr_multiple = 3.0
nifty_ma_window = 200
transaction_cost_percent = 0.001
slippage_percent = 0.001
trend_filter_threshold = 0.02 # Using the optimized parameter

# A list to store the returns from each individual trade across all stocks
all_portfolio_returns = []

# --- STEP 2: DOWNLOAD NIFTY 50 DATA FOR MARKET FILTER ---
try:
    nifty_df = yf.download('^NSEI', start=start_date, end=end_date, progress=False)
    nifty_df.columns = nifty_df.columns.droplevel(1)
    nifty_df['Nifty_MA'] = nifty_df['Close'].rolling(window=nifty_ma_window).mean()
    nifty_df.dropna(inplace=True)
    print("Nifty 50 data for market filter downloaded successfully.")
except Exception as e:
    print(f"Error downloading Nifty 50 data: {e}. Cannot proceed with market filter.")
    exit()

# --- STEP 3: MAIN BACKTESTING LOOP ---

for i, ticker in enumerate(tickers):
    # print(f"\n--- Backtesting {ticker} ({i+1}/{len(tickers)}) ---")

    # 3.1 Data Acquisition for the current ticker
    try:
        df = yf.download(ticker, start=start_date, end=end_date, progress=False)
        if df.empty:
            continue
        df.columns = df.columns.droplevel(1)
    except Exception as e:
        continue

    # 3.2 Define the "Mean", Deviation, and Indicators
    df['SMA'] = df['Close'].rolling(window=window).mean()
    df['StdDev'] = df['Close'].rolling(window=window).std()
    df['Upper_Band'] = df['SMA'] + (df['StdDev'] * 2)
    df['Lower_Band'] = df['SMA'] - (df['StdDev'] * 2)
    df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()
    df['Long_SMA'] = df['Close'].rolling(window=long_term_window).mean()
    df['ATR'] = ta.volatility.AverageTrueRange(high=df['High'], low=df['Low'], close=df['Close'], window=14).average_true_range()

    df = df.join(nifty_df[['Nifty_MA', 'Close']], how='inner', lsuffix='_stock', rsuffix='_nifty')
    df.rename(columns={'Close_nifty': 'Nifty_Close', 'Close_stock': 'Close'}, inplace=True)
    df.dropna(inplace=True)

    # 3.3 Implement the Trading Logic
    in_long_position = False
    in_short_position = False
    long_buy_price = 0
    short_sell_price = 0

    for j in range(len(df)):
        close_price = df['Close'][j]
        upper_band = df['Upper_Band'][j]
        lower_band = df['Lower_Band'][j]
        sma = df['SMA'][j]
        long_sma = df['Long_SMA'][j]
        rsi = df['RSI'][j]
        atr = df['ATR'][j]
        nifty_ma = df['Nifty_MA'][j]
        nifty_close = df['Nifty_Close'][j]

        # Individual stock trend filter
        is_trending_up = close_price > long_sma * (1 + trend_filter_threshold)
        is_trending_down = close_price < long_sma * (1 - trend_filter_threshold)

        # Corrected Market-wide trend filter
        nifty_is_bullish = nifty_close > nifty_ma
        nifty_is_bearish = nifty_close < nifty_ma

        # --- Long Position Logic ---
        if in_long_position and (close_price <= long_buy_price - (atr * atr_multiple)):
            profit = ((close_price - long_buy_price) / long_buy_price) - (2 * transaction_cost_percent) - (2 * slippage_percent)
            all_portfolio_returns.append(profit)
            in_long_position = False
        elif not in_long_position and not in_short_position and not is_trending_down and nifty_is_bullish and close_price < lower_band and rsi < 30:
            in_long_position = True
            long_buy_price = close_price
        elif in_long_position and close_price >= sma:
            profit = ((close_price - long_buy_price) / long_buy_price) - (2 * transaction_cost_percent) - (2 * slippage_percent)
            all_portfolio_returns.append(profit)
            in_long_position = False

        # --- Short Position Logic ---
        if in_short_position and (close_price >= short_sell_price + (atr * atr_multiple)):
            profit = ((short_sell_price - close_price) / short_sell_price) - (2 * transaction_cost_percent) - (2 * slippage_percent)
            all_portfolio_returns.append(profit)
            in_short_position = False
        elif not in_long_position and not in_short_position and not is_trending_up and nifty_is_bearish and close_price > upper_band and rsi > 70:
            in_short_position = True
            short_sell_price = close_price
        elif in_short_position and close_price <= sma:
            profit = ((short_sell_price - close_price) / short_sell_price) - (2 * transaction_cost_percent) - (2 * slippage_percent)
            all_portfolio_returns.append(profit)
            in_short_position = False

    if in_long_position:
        profit = ((df['Close'].iloc[-1] - long_buy_price) / long_buy_price) - transaction_cost_percent - slippage_percent
        all_portfolio_returns.append(profit)
    if in_short_position:
        profit = ((short_sell_price - df['Close'].iloc[-1]) / short_sell_price) - transaction_cost_percent - slippage_percent
        all_portfolio_returns.append(profit)

# --- STEP 4: EVALUATE PORTFOLIO RETURNS ---

print("\n\n--- Final Portfolio Backtest Complete ---")
if all_portfolio_returns:
    total_trades = len(all_portfolio_returns)
    cumulative_return = (pd.Series(all_portfolio_returns) + 1).prod() - 1
    win_rate = sum(r > 0 for r in all_portfolio_returns) / total_trades

    print(f"Total trades across all stocks: {total_trades}")
    print(f"Portfolio Cumulative Return: {cumulative_return:.2%}")
    print(f"Portfolio Win Rate: {win_rate:.2%}")
    print(f"Assumed Transaction Cost per trade: {transaction_cost_percent:.2%}")
    print(f"Assumed Slippage per trade: {slippage_percent:.2%}")
else:
    print("No trades were executed across the entire portfolio.")